# Data Encoding, Decoding and Flow

## Apache Thrift

The thrift type system includes base types like _bool, byte, double, string and integer_ but also special types like _binary_ and _struct_ (like classes) and also containers (_list, set, map_) that correspond to commonly available interfaces in most programming languages.

Base types:

- bool: A boolean value (true or false)
- byte: An 8-bit signed integer
- i16: A 16-bit signed integer
- i32: A 32-bit signed integer
- i64: A 64-bit signed integer
- double: A 64-bit floating point number
- string: A text string encoded using UTF-8 encoding

Thrift type definitions are defined in `.thrift` files. The Thrift compiler generates code in various languages from the `.thrift` files.

### Encoding

Let's use the following example record (JSON or dictionary-like) to encode:

```json
{
  "userName": "Martin",
  "favoriteNumber": 1337,
  "interests": ["daydreaming", "hacking"]
}
```

We can encode the record in Thrift using the following schema in the `.thrift` file:

```thrift
struct Person {
  1: required string userName,
  2: optional i64 favoriteNumber,
  3: optional list<string> interests
}
```

Thrift comes with a code generation tool that takes a schema definition like the ones shown here, and produces classes that implement the schema in various programming languages. Our code can call this generated code to encode or decode records of the schema.

The data encoded with this schema looks like this:
![thrift_binary_protocol](../assets/thrift_binary_protocol.png)

Each field has a type annotation (to indicate whether it is a string, integer, list, etc.) and, where required, a length indication (length of a string, number of items in a list). The strings that appear in the data (“Martin”, “daydreaming”, “hacking”) are encoded as UTF-8.

There are no field names (userName, favoriteNumber, interests). Instead, the encoded data contains _field tags_, which are numbers (1, 2, and 3). Those are the numbers that appear in the schema definition. Field tags are like aliases for fields—they are a compact way of saying what field we’re talking about, without having to spell out the field name.

Next, let's add a service. A service is a collection of method interfaces that can be called remotely. A service is defined in a `.thrift` file like this:

```thrift
service School {
    Person teachCourse(1: required Person person, 2: required string course)
}
```

The first line declares a service called `School`. The second line declares a method called `teachCourses`, which takes two arguments: a `Person` record and a `string`. The method returns a `Person` record.

### RPC

Nows, let's look at how to use the generated code to make remote procedure calls. We will write codes for 2 sides of the server-client application- the client initiates an RPC call and waits for a response from the server. The server executes the requested operation and returns a response to the client.

Here, we use `%%writefile` magic command to write the code to a file instead of running it in the cell.

In [1]:
%%writefile ../schema/person.thrift

struct Person {
  1: required string userName,
  2: optional i64 favoriteNumber,
  3: optional list<string> interests,
  4: optional byte grade
}

service School {
    Person teachCourse(1: required Person person, 2: required list<string> course)
    Person assignGrade(1: required Person person, 2: required byte grade)
}

Overwriting ../schema/person.thrift


In [2]:
%%writefile ../person_thrift_server.py
import thriftpy2
person_thrift = thriftpy2.load("./schema/person.thrift", module_name="person_thrift")

from thriftpy2.rpc import make_server

class School(object):
    def teachCourse(self, person, courses):
        for course in courses:
            person.interests.append(course)
        return person
    def assignGrade(self, person, grade):
        person.grade = grade
        return person

server = make_server(person_thrift.School, School(), client_timeout=None)
server.serve()

Overwriting ../person_thrift_server.py


Then, run `python person_thrift_server.py` in a new terminal. This will start the server.

In [4]:
import thriftpy2
person_thrift = thriftpy2.load("../schema/person.thrift", module_name="person_thrift")

from thriftpy2.rpc import make_client

school = make_client(person_thrift.School, timeout=None)

In [5]:
martin = person_thrift.Person(
    userName="Martin", favoriteNumber=1337, interests=["daydreaming", "hacking"]
)

In [6]:
martin.interests

['daydreaming', 'hacking']

In [12]:
martin = school.teachCourse(martin, ["coding"])

In [8]:
martin.interests

['daydreaming', 'hacking', 'c', 'o', 'd', 'i', 'n', 'g']

> 1. Add a new field `grade` (0-100) with an appropriate type annotation to the `Person` struct. Then, add a new method `assignGrade` to the `School` service that takes a `Person` record and a `grade` arguments, assigns the `grade` to the `Person` and returns the `Person`. Then call the method by passing `martin` and a grade number, and print his grade.
>
> 2. Add a method `teachCourses` to School to add a list of courses instead of just one course. Then pass `martin` and a list of course-- `["cooking", "sewing"]` to the method, and print his new interests.

In [9]:
martin = school.assignGrade(martin, 90)

In [13]:
martin

Person(userName='Martin', favoriteNumber=1337, interests=['daydreaming', 'hacking', 'c', 'o', 'd', 'i', 'n', 'g', 'cooking', 'sewing', 'coding'], grade=90)

In [11]:
martin = school.teachCourse(martin, ["cooking", "sewing"])

In [14]:
# Inspect the school client object and its available methods
print('School client type:', type(school))
print('Available methods:', dir(school))
# Example: Retrieve school object details (if any specific attributes exist)
try:
    print('School object:', school)
except Exception as e:
    print('Error retrieving school object:', e)

School client type: <class 'thriftpy2.thrift.TClient'>
Available methods: ['assignGrade', 'teachCourse']
School object: <thriftpy2.thrift.TClient object at 0x7824f414f2e0>
